<a href="https://colab.research.google.com/github/LeoTortega/Suicide-Text-Classification-GPT/blob/main/Suicide_Text_Classification_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Gathering

Buscando dados que serão utilizados para treino e teste do nosso modelo de classificação de texto.

In [ ]:
import pandas as pd
import gc

In [ ]:
# Carregando dataset com as mensagens que serão utilizadas para teste e treino
df = pd.read_csv("hf://datasets/Ram07/Detection-for-Suicide/detection_final_cleaned.csv")
df.drop('cleaned_text', axis=1, inplace=True)
df = df.head(10000)

In [ ]:
# Verificando as cinco primeiras linhas
df.head()

,class,text
0,suicide,Ex Wife Threatening SuicideRecently I left my ...
1,non-suicide,Am I weird I don't get affected by compliments...
2,non-suicide,Finally 2020 is almost over... So I can never ...
3,suicide,i need helpjust help me im crying so hard
4,suicide,It ends tonight.I can’t do it anymore. \nI quit.


# Limpeza dos dados
Nessa etapa limparemos a coluna "text" do nosso dataset. Aqui realizaremos a tokenização, identificaremos e limparemos as Stop Words que são desnecessárias e atrapalham o treinamento do modelo, e aplicaremos a lemantização.

In [ ]:
# Bibliotecas utilizadas para limpeza dos textos
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
# Tokenizer para dividir texto em palavras ou frases
nltk.download('punkt')
# Identifica e limpa as Stop Words do nosso dataset
nltk.download('stopwords')
# Aplica a lemantização
nltk.download('wordnet')

# Instanciando o lematizador
lemmatizer = WordNetLemmatizer()

# Carregar stop words em inglês
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Função para limpar o texto
def limpar_texto(texto):
  # Converter para letras minúsculas
  texto = texto.lower()

  # Remover caracteres especiais
  texto = re.sub(r'[^\w\s]', '', texto)

  # Remover números
  texto = re.sub(r'\d+', '', texto)

  # Tokenizar o texto
  tokens = word_tokenize(texto)

  # Remover Stop Words e lematizar
  tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

  # Reconstruir o texto limpo
  texto_limpo = ' '.join(tokens)

  return texto_limpo

In [ ]:
df['cleaned_text'] = df['text'].apply(limpar_texto)
df.head()

,class,text,cleaned_text
0,suicide,Ex Wife Threatening SuicideRecently I left my ...,ex wife threatening suiciderecently left wife ...
1,non-suicide,Am I weird I don't get affected by compliments...,weird dont get affected compliment coming some...
2,non-suicide,Finally 2020 is almost over... So I can never ...,finally almost never hear bad year ever swear ...
3,suicide,i need helpjust help me im crying so hard,need helpjust help im cry hard
4,suicide,It ends tonight.I can’t do it anymore. \nI quit.,end tonighti cant anymore quit


# Treinando Modelo


In [ ]:
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer

In [ ]:
# Separando colunas para treino e teste
x = df['cleaned_text']
y = df['class'].apply(lambda x: 1 if x == 'suicide' else 0)  # Converte para 1 e 0

# Dividindo o dataset em treino e teste utilizando estratificação
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y)

# Verificar os tamanhos dos conjuntos
print(f'Tamanho do conjunto de treino: {len(x_train)}')
print(f'Tamanho do conjunto de teste: {len(x_test)}')

Tamanho do conjunto de treino: 8000
Tamanho do conjunto de teste: 2000


In [ ]:
# Removendo DF da memória
del df
del x
del y

gc.collect()

0

In [ ]:
# Carregar o tokenizer do GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# Reutilizar o eos_token como pad_token
tokenizer.pad_token = tokenizer.eos_token

# Tokenizar o texto de treino
train_encodings = tokenizer(list(x_train), truncation=True, padding=True)
test_encodings = tokenizer(list(x_test), truncation=True, padding=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Aqui iremos criar um dataset no formato PyTorch, para que consigamos treinar nosso modelo GPT-2 do HuggingFace. Essa é uma estrutura que é comumente usada com o Hugging Face Transformers

In [ ]:
import torch
from torch.utils.data import Dataset

class SuicideDataset(Dataset):
    def __init__(self, encodings, labels):
        # O dataset recebe os textos tokenizados (encodings) e os rótulos (labels)
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        # Retorna o tamanho do dataset
        return len(self.labels)

    # Método utilizado para alimentar o modelo durante o treinamento
    def __getitem__(self, idx):
        # Retorna um item do dataset
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [ ]:
# Criando dataset de treino e teste
train_dataset = SuicideDataset(train_encodings, y_train.tolist())
test_dataset = SuicideDataset(test_encodings, y_test.tolist())

In [ ]:
# Limpando dados desnecessários da memoria
del train_encodings
del test_encodings
gc.collect()

0

In [ ]:
from transformers import GPT2ForSequenceClassification, Trainer, TrainingArguments

# Carregar o modelo GPT-2 para classificação de sequência
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)

# Definir um token de preenchimento
model.config.pad_token_id = tokenizer.eos_token_id  # Use o token de fim de sequência como token de preenchimento

# Definir os argumentos de treinamento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Criar o objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Treinar o modelo
trainer.train()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.382900
1000,0.243900
1500,0.159700
2000,0.121100
2500,0.047700
3000,0.075400


TrainOutput(global_step=3000, training_loss=0.17178062121073404, metrics={'train_runtime': 5869.7451, 'train_samples_per_second': 4.089, 'train_steps_per_second': 0.511, 'total_flos': 1.2542244028416e+16, 'train_loss': 0.17178062121073404, 'epoch': 3.0})

In [ ]:
# Salvar modelo e tokenizer no Colab
model.save_pretrained("meu_modelo_gpt2_classificador")
tokenizer.save_pretrained("meu_modelo_gpt2_classificador")

('meu_modelo_gpt2_classificador/tokenizer_config.json',
 'meu_modelo_gpt2_classificador/special_tokens_map.json',
 'meu_modelo_gpt2_classificador/vocab.json',
 'meu_modelo_gpt2_classificador/merges.txt',
 'meu_modelo_gpt2_classificador/added_tokens.json')

In [ ]:
!zip -r meu_modelo_gpt2_classificador.zip meu_modelo_gpt2_classificador/

  adding: meu_modelo_gpt2_classificador/ (stored 0%)
  adding: meu_modelo_gpt2_classificador/special_tokens_map.json (deflated 74%)
  adding: meu_modelo_gpt2_classificador/config.json (deflated 53%)
  adding: meu_modelo_gpt2_classificador/vocab.json (deflated 68%)
  adding: meu_modelo_gpt2_classificador/model.safetensors (deflated 7%)
  adding: meu_modelo_gpt2_classificador/merges.txt (deflated 53%)
  adding: meu_modelo_gpt2_classificador/tokenizer_config.json (deflated 55%)


In [ ]:
from google.colab import files
files.download("meu_modelo_gpt2_classificador.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Use o token de EOS como token de preenchimento

# Função de predição movendo entradas para o dispositivo correto
def predict_class(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs para a GPU
    outputs = model(**inputs)
    prediction = outputs.logits.argmax(-1).item()
    return prediction

# Exemplo de predição
texto_exemplo = "My life is miserable"
print(predict_class(texto_exemplo))

0


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = [predict_class(text) for text in x_test]

accuracy = accuracy_score(y_test, y_pred)  # y_test sendo os rótulos reais
print(f'Acurácia: {accuracy * 100:.2f}%')

Acurácia: 98.80%


In [ ]:
# Exemplo de predição
texto_exemplo = "dont want die dont want live either kinda stuck..."
print(predict_class(texto_exemplo))

1


In [ ]:
x_test.head(10)

,cleaned_text
4819,german accent gtgtgtgtgt accent seriously r fu...
9526,wish good friend life guess huh
6221,dont want die dont want live eitherkinda stuck...
799,stop feeling stupid always feel stupid around ...
1680,yeah im listeing music playing super mario ody...
648,id love get full body laser hair removal im ol...
3388,know shouldnt kill myselfi know shouldnt kill ...
429,transform christmas something nonreligious yeah
3557,james earl jones james charles kid james earl ...
4817,im tired livinglets talk bit tell youre intera...
